## Importación de módulos

In [1]:
import requests
import numpy as np
import pandas as pd
import quandl
import warnings
warnings.filterwarnings("ignore")

## Descarga históricos de BitStamp

In [2]:
def fetch_bitstamp_data(symbol, start, end, timeframe, limit=1000):
    url = f"https://www.bitstamp.net/api/v2/ohlc/{symbol}/"
    data_frames = []
    while start < end:
        # Ajustar end para la solicitud actual para no exceder el límite de 1000 registros
        current_end = start + (timeframe * limit)
        # Debug
        #print(f"{pd.to_datetime(start, unit='s')} - {pd.to_datetime(current_end, unit='s')}")
        params = {
            'start': int(start),
            'end': int(current_end),
            'step': timeframe,
            'limit': limit,
            'exclude_current_candle': False
        }
        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                data = response.json()
                df = pd.DataFrame(data['data']['ohlc'])
                if not df.empty:
                    data_frames.append(df)
            else:
                raise Exception(f"Failed to fetch data: {response.status_code}, {response.text}")
        except Exception as e:
            print(e)
            break
        start = current_end
    # Combinar todos los DataFrames
    if data_frames:
        df = pd.concat(data_frames, ignore_index=True)
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
        df = df.set_index('timestamp')
        df = df.sort_index()
        df.index.name = 'date'
        df = df.astype({
            'open': float,
            'high': float,
            'low': float,
            'close': float,
            'volume': float
        })
        # Comprobar filas duplicadas
        if df.index.duplicated().any():
            df = df[~df.index.duplicated(keep='first')]
        return df
    else:
        return pd.DataFrame()

## Descarga de métricas blockchain de BTC

In [ ]:
def fetch_quandl_bchain_data():
    # API de Quandl
    quandl.ApiConfig.api_key = 'soxd-469x3Zp4ib_4uzc'
    # Lista de códigos de las métricas
    metrics_codes = [
        'MKPRU', 'MWNUS', 'HRATE', 'DIFF', 'MKPRU', 'MIREV', 'CPTRA',
        'TRVOU', 'CPTRV', 'ETRVU', 'ETRAV', 'TOUTV', 'NTRBL', 'NTRAT',
        'NADDU', 'NTREP', 'NTRAN', 'TRFUS', 'TRFEE', 'MKTCP', 'TOTBC',
        'MWNTD', 'MWTRV'
    ]
    # Diccionario para almacenar los dataframes
    dfs = {}
    for code in metrics_codes:
        # Obtener la tabla de Quandl para cada métrica
        df = quandl.get_table('QDL/BCHAIN', code=code)
        # Almacenar el DataFrame en el diccionario con el código como clave
        dfs[code] = df.set_index('date')[['value']].rename(columns={'value': code})
    # Concatenar todos los DataFrames a lo largo del eje de las columnas
    return pd.concat(dfs.values(), axis=1, join='outer')
    """
    # Calcular las medias móviles
    df['SMA30'] = df['HRATE'].rolling(window=30).mean()
    df['SMA60'] = df['HRATE'].rolling(window=60).mean()
    # Detectar señales de capitulación y recuperación
    df['Capitulation'] = df['SMA30'] < df['SMA60']
    df['Buy_Signal'] = (df['SMA30'] > df['SMA60']) & (df['Capitulation'].shift(1) == True)
    df['Sell_Signal'] = (df['SMA30'] < df['SMA60']) & (df['Capitulation'].shift(1) == False)
    # Visualización
    plt.figure(figsize=(14, 7))
    #plt.plot(df.index, df['HRATE'], label='Hash Rate')
    plt.plot(df.index, df['SMA30'], label='30-Day SMA', color='orange')
    plt.plot(df.index, df['SMA60'], label='60-Day SMA', color='green')
    plt.scatter(df.index[df['Buy_Signal']], df['SMA30'][df['Buy_Signal']], color='green', label='Buy Signal', marker='^', s=100)
    plt.scatter(df.index[df['Sell_Signal']], df['SMA30'][df['Sell_Signal']], color='red', label='Sell Signal', marker='v', s=100) 
    plt.title('Hash Rate and Moving Averages')
    plt.legend()
    plt.show()
    """

## Marca de tiempo de máximos y mínimos

In [3]:
def find_timestamp_extremum(df_highest, df_lowest):
    """
    params:
        df_highest_timeframe(highest timeframe OHLCV dataframe)
        df_lowest_timeframe(lowest timeframe OHLCV dataframe)
    """
    # Añadir nuevas columnas si no existen
    for col in ['Low_time', 'High_time', 'First']:
        if col not in df_highest.columns:
            df_highest[col] = np.nan
    # Establecer qué sucedió antes si High o Low
    index_starts = df_highest.index[:-1]
    index_ends = df_highest.index[1:]
    for start, end in zip(index_starts, index_ends):
        row_lowest_timeframe = df_lowest.loc[start:end]
        if len(row_lowest_timeframe) > 1:
            row_lowest_timeframe = row_lowest_timeframe.iloc[:-1]
        try:
            high = row_lowest_timeframe['high'].idxmax()
            low = row_lowest_timeframe['low'].idxmin()
            df_highest.loc[start, 'High_time'] = high
            df_highest.loc[start, 'Low_time'] = low
        except Exception as e:
            print(f'Exception occurred: {e}')
            df_highest.loc[start, 'High_time'] = start
            df_highest.loc[start, 'Low_time'] = start
    # Asegurar que las columnas son de tipo datetime
    df_highest['High_time'] = pd.to_datetime(df_highest['High_time'])
    df_highest['Low_time'] = pd.to_datetime(df_highest['Low_time'])
    # Find out which appears first
    df_highest.loc[df_highest['High_time'] > df_highest['Low_time'], 'First'] = 1
    df_highest.loc[df_highest['High_time'] < df_highest['Low_time'], 'First'] = 2
    df_highest.loc[df_highest['High_time'] == df_highest['Low_time'], 'First'] = 0
    # Verificar el número de filas sin TP ni SL al mismo tiempo
    percentage_garbage_row = len(df_highest.loc[df_highest['First']==0].dropna()) / len(df_highest) * 100.
    print(f'WARNING: Filas no válidas para establecer TP/SL: {percentage_garbage_row:.2f} %')
    # Remover la última columna porque no es posible encontrar el extremo
    df_highest = df_highest.iloc[:-1]
    return df_highest

## Cálculo de TP y SL

In [4]:
def run_tp_sl(df, df_lowest, leverage=1, tp=0.015, sl=-0.015, cost=0.00):
    '''
    params (mandatory): 
        df - DataFrame with High_time and Low_time columns
    params (optional):
        leverage=1, tp_0.015, ls=-0.015, cost=0.00
    return:
        df - Incoming DataFrame with two new columns ['returns', 'duration']
    '''
    # Encontrar marca de tiempo de máximos y mínimos
    df = find_timestamp_extremum(df, df_lowest)
    # El trade comienza en la SIGUIENTE vela
    signal_column = df['signal']
    df['signal'] = df['signal'].shift(1).fillna(0)
    # Establecer valores iniciales
    buy = False
    sell = False
    df['duration'] = 0
    # Bucle principal
    for i in range(len(df)):
        # Extraer fila
        row = df.iloc[i]
        # ABRIR COMPRA
        if buy == False and row['signal'] == 1:
            buy = True
            open_buy_price = row['open']
            open_buy_date = row.name
        if buy:
            # Verificar variación
            var_buy_high = (row['high'] - open_buy_price) / open_buy_price
            var_buy_low = (row['low'] - open_buy_price) / open_buy_price
            # Comprobar localización de TP y SL
            if (var_buy_high > tp) and (var_buy_low < sl):
                # Si ambos tienen el mismo timestamp, no se considera el trade
                if row["First"] == 0:
                    pass
                elif row['First'] == 2:
                    df.loc[row.name, 'returns'] = (tp - cost) * leverage
                    df.loc[row.name, 'duration'] = row['High_time'] - open_buy_date
                elif row['First'] == 1:
                    df.loc[row.name, 'returns'] = (sl - cost) * leverage
                    df.loc[row.name, 'duration'] = row['Low_time'] - open_buy_date
                # Resetear valores
                buy = False
                open_buy_price = None
                var_buy_high = 0
                var_buy_low = 0
                open_buy_date = None
            elif var_buy_high > tp:
                df.loc[row.name, 'returns'] = (tp - cost) * leverage
                df.loc[row.name, 'duration'] = row['High_time'] - open_buy_date
                # Resetear valores
                buy = False
                open_buy_price = None
                var_buy_high = 0
                var_buy_low = 0
                open_buy_date = None
            elif var_buy_low < sl:
                df.loc[row.name, 'returns'] = (sl - cost) * leverage
                df.loc[row.name, 'duration'] = row['Low_time'] - open_buy_date
                # Resetear valores
                buy = False
                open_buy_price = None
                var_buy_high = 0
                var_buy_low = 0
                open_buy_date = None
        # ABRIR VENTA
        if sell == False and row['signal'] == -1:
            sell = True
            open_sell_price = row['open']
            open_sell_date = row.name
        if sell:
            # Verificar variación
            var_sell_high = -(row['high'] - open_sell_price) / open_sell_price
            var_sell_low = -(row['low'] - open_sell_price) / open_sell_price
            # Comprobar localización de TP y SL
            if (var_sell_low > tp) and (var_sell_high < sl):
                # Si ambos tienen el mismo timestamp, no se considera el trade
                if row['First'] == 0:
                    pass
                elif row['First'] == 1:
                    df.loc[row.name, 'returns'] = (tp - cost) * leverage
                    df.loc[row.name, 'duration'] = row['Low_time'] - open_sell_date
                elif row['First'] == 2:
                    df.loc[row.name, 'returns'] = (sl - cost) * leverage
                    df.loc[row.name, 'duration'] = row['High_time'] - open_sell_date
                # Resetear valores
                sell = False
                open_sell_price = None
                var_sell_high = 0
                var_sell_low = 0
                open_sell_date = None
            elif var_sell_low > tp:
                df.loc[row.name, 'returns'] = (tp - cost) * leverage
                df.loc[row.name, 'duration'] = row['Low_time'] - open_sell_date
                # Resetear valores
                sell = False
                open_sell_price = None
                var_sell_high = 0
                var_sell_low = 0
                open_sell_date = None
            elif var_sell_high < sl:
                df.loc[row.name, 'returns'] = (sl - cost) * leverage
                df.loc[row.name, 'duration'] = row['High_time'] - open_sell_date
                # Resetear valores
                sell = False
                open_sell_price = None
                var_sell_high = 0
                var_sell_low = 0
                open_sell_date = None
    # Eliminar columnas inncesarias
    df = df.drop(labels=['Low_time', 'High_time'], axis=1)
    # Rellenar con 0s valores faltantes
    df['returns'] = df['returns'].fillna(value=0)
    # Reestablecer señal
    df['signal'] = signal_column
    # retornar df
    return df